Notebook to quickly get infos on the saved weights (model, loss, etc.)

Not structured but just what I needed for quick checks, helped me sort the weights I've trained (which were a lot.)

In [108]:
import torch
from util.handle_files import * 
import pandas as pd
import seaborn as sns
def load(path):
    """
    Loads the checkpoint
    """
    checkpoint = torch.load(path, map_location=lambda storage, loc: storage)
    return checkpoint

def stats(ckpt):
    """
    gets the stats from a torch checkpoint
    """
    epochs = ckpt['epoch']
    args = ckpt['args']
    bvl = ckpt['best_val_loss']
    #print('EPOCH {}\n'.format(epochs))
    #print('Args', args)
    #print('\nBEST VAL LOSS', bvl)
    return epochs, args, bvl
    
def full(path):
    """
    Prints stats and returns them for the df
    """
    #print('#############################')
    #print('\nWeight name : ', path)
    x = load(path)
    e,a,b = stats(x)
    #print('--------------------------------\n')
    return e, a, b
    
def list_best(folderpath):
    """
    full pipeline
    """
    lw = list_files(folderpath+'*', '.tar')
    df = pd.DataFrame()
    for z in filter(lambda x: 'best' in x, lw):
        e,a,b = full(z)
        x = z.split('\\')
        name = x[1]+'/'+x[2]
        df = df.append({'wt': name, 'epoch':e,'args':a,'loss':b}, ignore_index=True)
    return df.sort_values('loss')#.query('loss<=0.05')

def list_all(folderpath):
    """
    full pipeline
    """
    lw = list_files(folderpath+'*', '.tar')
    df = pd.DataFrame()
    for z in lw:
        e,a,b = full(z)
        x = z.split('\\')
        name = x[1]+'/'+x[2]
        df = df.append({'wt': name, 'epoch':e,'args':a,'loss':b}, ignore_index=True)
    return df.sort_values('loss').query('loss<=0.05')

def check_arch(path):
    """ Check to make sure I have the right weights with the right architecture names"""
    x = load(path)
    print(path)
    for param_tensor in x['state_dict']:
        print(param_tensor, "\t", x['state_dict'][param_tensor].size())
        
def full_check(path, df, index):
    check_arch(path+df.loc[index]['wt'])
    print("\n",df.loc[index]['args'])

In [103]:
check_arch('./trained_models/wormbrain_basefr/affine_wormbrain1_basefr.pth.tar')
check_arch('./trained_models/wormbrain_basefr/affine_wormbrain1_basefr.pth.tar')

./trained_models/wormbrain_basefr/affine_wormbrain1_basefr.pth.tar
FeatureExtraction.model.0.weight 	 torch.Size([32, 3, 5, 5])
FeatureExtraction.model.0.bias 	 torch.Size([32])
FeatureExtraction.model.1.weight 	 torch.Size([32])
FeatureExtraction.model.1.bias 	 torch.Size([32])
FeatureExtraction.model.1.running_mean 	 torch.Size([32])
FeatureExtraction.model.1.running_var 	 torch.Size([32])
FeatureExtraction.model.1.num_batches_tracked 	 torch.Size([])
FeatureExtraction.model.3.weight 	 torch.Size([64, 32, 5, 5])
FeatureExtraction.model.3.bias 	 torch.Size([64])
FeatureExtraction.model.4.weight 	 torch.Size([64])
FeatureExtraction.model.4.bias 	 torch.Size([64])
FeatureExtraction.model.4.running_mean 	 torch.Size([64])
FeatureExtraction.model.4.running_var 	 torch.Size([64])
FeatureExtraction.model.4.num_batches_tracked 	 torch.Size([])
FeatureExtraction.model.7.weight 	 torch.Size([128, 64, 5, 5])
FeatureExtraction.model.7.bias 	 torch.Size([128])
FeatureExtraction.model.8.weight 	 t

In [109]:
path = './trained_models/test_arch/fullset/'
df_test_arch = list_best(path)
df_test_arch.head()

,args,epoch,loss,wt
0,"Namespace(batch_size=16, dataset_csv_path='tra...",37.0,0.020444,AffineWB_dropout02/best_rgb512_aug_AffineWB_dr...
7,"Namespace(batch_size=16, dataset_csv_path='tra...",49.0,0.037828,best_newFR/best_rgb512_aug_TPS_test_2_newFR_dr...
6,"Namespace(batch_size=16, dataset_csv_path='tra...",49.0,0.037828,TPS_test_2_newFR_dropout015xd/best_rgb512_aug_...
8,"Namespace(batch_size=16, dataset_csv_path='tra...",51.0,0.037828,best_newFR/rgb512_aug_TPS_test_2_newFR_dropout...
5,"Namespace(batch_size=16, dataset_csv_path='tra...",30.0,0.038993,TPS_test_2_newFR_dropout015/best_rgb512_aug_TP...


In [110]:
full_check(path,df_test_arch, 7)

./trained_models/test_arch/fullset/best_newFR/best_rgb512_aug_TPS_test_2_newFR_dropout015xd_tps_grid_losstest_2.pth.tar
FeatureExtraction.model.0.weight 	 torch.Size([32, 3, 5, 5])
FeatureExtraction.model.0.bias 	 torch.Size([32])
FeatureExtraction.model.1.weight 	 torch.Size([32])
FeatureExtraction.model.1.bias 	 torch.Size([32])
FeatureExtraction.model.1.running_mean 	 torch.Size([32])
FeatureExtraction.model.1.running_var 	 torch.Size([32])
FeatureExtraction.model.1.num_batches_tracked 	 torch.Size([])
FeatureExtraction.model.3.weight 	 torch.Size([64, 32, 5, 5])
FeatureExtraction.model.3.bias 	 torch.Size([64])
FeatureExtraction.model.4.weight 	 torch.Size([64])
FeatureExtraction.model.4.bias 	 torch.Size([64])
FeatureExtraction.model.4.running_mean 	 torch.Size([64])
FeatureExtraction.model.4.running_var 	 torch.Size([64])
FeatureExtraction.model.4.num_batches_tracked 	 torch.Size([])
FeatureExtraction.model.7.weight 	 torch.Size([128, 64, 5, 5])
FeatureExtraction.model.7.bias 	 t

In [115]:
simpler = './trained_models/simpler/'
df_simpler = list_best(simpler)
df_simpler

,args,epoch,loss,wt
0,"Namespace(batch_size=16, dataset_csv_path='tra...",40.0,0.008201,affineWB1_simplerScheduler1e7/best_rgb512_aug_...
1,"Namespace(batch_size=16, dataset_csv_path='tra...",31.0,0.028545,tpsWB1_simplerAnother/best_rgb512_aug_tpsWB1_s...
4,"Namespace(batch_size=16, dataset_csv_path='tra...",24.0,0.029655,tpsWB1_simpler/best_rgb512_aug_tpsWB1_simpler_...
2,"Namespace(batch_size=16, dataset_csv_path='tra...",21.0,0.031329,tpsWB1_simplerLastTry/best_rgb512_aug_tpsWB1_s...
3,"Namespace(batch_size=16, dataset_csv_path='tra...",23.0,0.031951,tpsWB1_simplerScheduler1e7/best_rgb512_aug_tps...


In [125]:
full_check(simpler,df_simpler,1)

./trained_models/simpler/tpsWB1_simplerAnother/best_rgb512_aug_tpsWB1_simplerAnother_tps_grid_losswormbrain_1.pth.tar
FeatureExtraction.model.0.weight 	 torch.Size([32, 3, 5, 5])
FeatureExtraction.model.0.bias 	 torch.Size([32])
FeatureExtraction.model.1.weight 	 torch.Size([32])
FeatureExtraction.model.1.bias 	 torch.Size([32])
FeatureExtraction.model.1.running_mean 	 torch.Size([32])
FeatureExtraction.model.1.running_var 	 torch.Size([32])
FeatureExtraction.model.1.num_batches_tracked 	 torch.Size([])
FeatureExtraction.model.3.weight 	 torch.Size([64, 32, 5, 5])
FeatureExtraction.model.3.bias 	 torch.Size([64])
FeatureExtraction.model.4.weight 	 torch.Size([64])
FeatureExtraction.model.4.bias 	 torch.Size([64])
FeatureExtraction.model.4.running_mean 	 torch.Size([64])
FeatureExtraction.model.4.running_var 	 torch.Size([64])
FeatureExtraction.model.4.num_batches_tracked 	 torch.Size([])
FeatureExtraction.model.7.weight 	 torch.Size([128, 64, 5, 5])
FeatureExtraction.model.7.bias 	 tor